In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order

import datetime
import time
import threading

In [ ]:
init_event = threading.Event() # init event
id_event = threading.Event()
hist_event = threading.Event()
order_event = threading.Event()
evec_event = threading.Event()
port_event = threading.Event()
value_event = threading.Event()

In [ ]:
# define the App class
class App(EClient, EWrapper):
    def __init__(self, address, port, cid):
        EClient.__init__(self, self)
        # # list to store data
        self.bar_dict = {}
        # create a connection with IBKR
        self.connect(address, port, cid)
        # start client
        thread = threading.Thread(target=self.run)
        thread.start()
        init_event.set()


    def nextValidId(self, orderId: int):
        # provide a new order id for each of my requests
        super().nextValidId(orderId)
        logging.debug("setting nextValidOrderId: %d", orderId)
        self.nextValidOrderId = orderId
        print("NextValidId:", orderId)
        id_event.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.bar_dict.keys():
            self.bar_dict[reqId] = []
        self.bar_dict[reqId].append(vars(bar))
        
    def historicalDataEnd(self, reqId, start, end):
        print(f"end of historicalData")
        hist_event.set()


    # implement code to monitor trade status and receive confirmation of the trade
    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderstate: OrderState):
        # openorder callback
        print(f"openOrder. orderId:{orderId}, contract:{contract}, order:{order}")
        order_event.set()

    def orderStatus(self, orderId: OrderId, status: str, filled: float, reamining: float, avgFillPrice: float,
                    permId: int, parenId: int, lastFillPrice: float, clientId: int, whyHeld:str, mktCapPrice: float):
                    # orderstatus callback
        print(f"orderStatus. orderId: {orderId}, status: {status}, filled: {filled}, remaining:{reamining}, avgFillPrice: {avgFillPrice}, permId:{permId}, parentId:{parenId}, lastFillPrice: {lastFillPrice}, clientId: {clientId}, whyHeld: {whyHeld}, mktCapPrice:{mktCapPrice}")

    def execDetails(self, reqId: int, contract: Contract, execution: Execution):
        print(f"execDetails. reqId: {reqId}, contract: {contract}, execution: {execution}")
        evec_event.set()
    # basically a summary

    def commissionReport(self, commissionReport: CommissionReport):
        super().commissionReport(commissionReport)
        print("CommissionReport.", commissionReport)
        
    # called when query portfolio information
    def updatePortfolio(self, contract, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL, accountName):
        self.last_portfolio = pd.DataFrame(columns=["ticker","quantity","marketPrice","marketValue","averageCost","unrealizedPNL","realizedPNL"])
        self.last_portfolio = self.last_portfolio.append({"ticker":contract.symbol,"quantity":position,"marketPrice":marketPrice,"marketValue":marketValue,"averageCost":averageCost,"unrealizedPNL":unrealizedPNL,"realizedPNL":realizedPNL}, ignore_index=True)
        port_event.set()

    # query total value
    def updateAccountSummary(self, reqId: int, account: str, tag: str, value: str, currency: str):
        if tag == 'NetLiquidation':
            self.total_value = float(value)
            print("Total value of the account: ", self.total_value)
        value_event.set()

    def reqIds(self):
        super().reqIds(-1)
        id_event.wait()
        id_event.clear()

In [ ]:
# Connect to the TWS API
client = EClient(EWrapper())
client.connect("127.0.0.1", 7497, 0)

# Get the S&P 500 index information
sp500 = yf.Ticker("^GSPC")

# Retrieve the components of the index
sp500_components = sp500.info['components']

# Extract the ticker symbols of the components
sp500_tickers = [component['symbol'] for component in sp500_components]

# Pull in historical data for the S&P 500 components using yfinance
data = yf.download(sp500_tickers, start="2010-01-01", end="2022-12-31")

#TODO: note we might also need to get data from tws.


# main part
# strategy implementation
* decide what to buy & sell, as well as the quantity

下面这个cell不重要, 需要改写

In [ ]:
#TODO: rewrite this
# Calculate the UMD, BAB, and QMJ factors
data["UMD"] = data["Close"] / data["Close"].rolling(252).mean()
data["BAB"] = data["Volume"] / data["Volume"].rolling(252).mean()
data["QMJ"] = data["Close"].rolling(252).std()

# Calculate the Momentum factor
data["Momentum"] = data["Close"].pct_change(252)

# Create a trading algorithm that uses the momentum factor to generate buy and sell signals
# Buy a component when Momentum > 0.05
# Sell a component when Momentum < -0.05

# query the current portfolio


In [ ]:
app.reqAccountUpdates(True, "DU222526") #TODO: replace the DU222526 with our account ID
port_event.wait()
port_event.clear()

app.reqAccountSummary(1, "All", "NetLiquidation")
value_event.wait()
value_event.clear()

last_portfolio = app.last_portfolio
total_value = app.total_value

# decide the target portfolio
* assume we know the weight of the target_portfolio
* total_value - 1000 to save 1000 dollars in case something goes wrong

In [ ]:
#TODO: we need to get the weight, mkt_price, ticker from main part of this notebook
target_portfolio['quantity'] = np.floor(target_portfolio['weight'] * (total_value - 1000) / target_portfolio['mkt_price'])

# place order

* target_portfolio
> target_portfolio python pandas dataframe containing information of stocks portfolio(target portfolio) that I want to form through trading. The first column of the dataframe is stocks' tickers and the second column of the dataframe is the quantity of each stock that I want to hold. 

* last_portfolio
> last_portfolio is a dataframe containing information of last portfolio that I hold

In [ ]:
# Create a list to store the stocks that need to be sold
stocks_to_sell = []

# Iterate through the target_portfolio dataframe
for index, row in target_portfolio.iterrows():
    # Check if the stock is present in the last_portfolio dataframe
    last_portfolio_stock = last_portfolio[last_portfolio['ticker'] == row['ticker']]
    if not last_portfolio_stock.empty:
        # Compare the quantities
        last_portfolio_quantity = last_portfolio_stock['quantity'].values[0]
        if row['quantity'] < last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = last_portfolio_quantity - row['quantity'] 
            # Add the stock to the list of stocks to sell
            stocks_to_sell.append((row['ticker'], quantity_diff))

for stock in stocks_to_sell:
    # Replace the place holder values with the appropriate values
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "SELL"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    app.reqIds() # require a new id
    orderid = app.nextValidOrderId
    app.placeOrder(orderid, contract, order)
